### Set up

In [ ]:
#devtools::install_github('allogamous/EnvRtype',force=TRUE) # current version:  1.1.0 (June 2022)

In [ ]:
library("tidyr")
library("dplyr")
library("ggplot2")
library("devtools")
library("EnvRtype")

### Example data

In [ ]:
data("maizeYield") # toy set of phenotype data (grain yield per environment)
data("maizeG"    ) # toy set of genomic relationship for additive effects
data("maizeWTH")   # toy set of environmental data
y   = "value"      # name of the vector of phenotypes
gid = "gid"        # name of the vector of genotypes
env = "env"        # name of the vector of environments
maizeYield <- droplevels(maizeYield) # make sure the nlevels of GID matchs with the G matrix dimension

In [ ]:
## environment, genotype, trait value (grain yield)
head(maizeYield)

750 records from **150 genotypes** in **5 environments**:

In [ ]:
table(maizeYield$env)

In [ ]:
head(maizeWTH)

In [ ]:
dim(maizeWTH)

`W_matrix` calculates a **matrix of environmental covariables**: average `k` envirotyping descriptors for `q` environments

In [ ]:
W  = W_matrix(env.data = maizeWTH, var.id = c("FRUE",'PETP',"SRAD","T2M_MAX"), statistic = 'mean')

In [ ]:
W

In [ ]:
## KG and KE might be a list of kernels
KE = list(W = env_kernel(env.data = W)[[2]])
KG = list(G=maizeG);

##### Environmental relatedness matrix (k x k)

In [ ]:
KE$W

##### Genetic relatedness matrix (g x g)

In [ ]:
KG$G[1:5, 1:5]

### Kernel models (EnvRtype)

In [ ]:
writeLines(" - CREATE KERNEL MODELS")
## Creating kernel models with get_kernel
MM    = get_kernel(K_G = KG, y = y, gid = gid, env = env, data = maizeYield,model = "MM")
MDs   = get_kernel(K_G = KG, y = y,gid = gid,env = env,  data = maizeYield, model = "MDs")
EMM   = get_kernel(K_G = KG, K_E = KE, y = y,gid = gid,env = env,  data = maizeYield, model = "EMM")
EMDs  = get_kernel(K_G = KG, K_E = KE, y = y,gid = gid,env = env,  data = maizeYield, model = "EMDs")
RMMM  = get_kernel(K_G = KG, K_E = KE, y = y,gid = gid,env = env,  data = maizeYield, model = "RNMM")
RNMDs = get_kernel(K_G = KG, K_E = KE, y = y,gid = gid,env = env,  data = maizeYield, model = "RNMDs")

In [ ]:
writeLines(" - FIT KERNEL MODELS")
fixed = model.matrix(~0+env, maizeYield)
fit   = kernel_model(y = y,env = env,gid = gid, data = maizeYield,random = EMDs,fixed = fixed)

In [ ]:
print(fit$yHat)    # predicted phenotype values

In [ ]:
print("Variance Components")
print(fit$VarComp) # variance components and confidence intervals

print("full model output")
print(fit$BGGE)    # full output of Hierarchical Bayesian Modeling

### Data for ASPA 2025

In [ ]:
library("data.table")

##### Phenotypes

In [ ]:
url = "http://www.jackdellequerce.com/data/enviromics/pheno.csv"

In [ ]:
phenotypes <- fread(url)

In [ ]:
head(phenotypes)

In [ ]:
Y = phenotypes |>
  gather(key = "environment", value = "milk", -sire)

Y = na.omit(Y)
Y$environment = as.factor(Y$environment)
Y$sire = as.factor(Y$sire)

head(Y)

In [ ]:
dim(Y)

##### Covariates

In [ ]:
url = "http://www.jackdellequerce.com/data/enviromics/covariates.csv"
covariates = fread(url)
head(covariates)

##### Environmental descriptors (envirotyping)

In [ ]:
url = "http://www.jackdellequerce.com/data/enviromics/enviro_typing.csv"
envtypes = fread(url)
head(envtypes)

In [ ]:
dim(envtypes)

#### Environmental relatedness matrix

Given $\mathbf{W}$ (e x q), a matrix of `q` environmental descriptors (e.g. max temperature, average humidity etc.) on `e` environments, we calculate the **environmental relatedness matrix** as:

$$
K_E = \frac{WW'}{\text{trace}(WW')/\text{nrow}(W)}
$$

In [ ]:
## function to scale data, if needed
scale_this <- function(x){
  (x - mean(x, na.rm=TRUE)) / sd(x, na.rm=TRUE)
}

In [ ]:
E <- envtypes[,-1]
temp <- E |>
  group_by(prov) |>
  #mutate_all(scale_this) |>
  #group_by(prov) |>
  summarise_all(mean)

print("average descriptors by environment")
env_names = temp$prov
temp <- as.matrix(temp[,-1])
rownames(temp) <- env_names
temp

In [ ]:
## The env_kernel function has two outputs called varCov (relatedness among covariables) and envCov (relatedness among environments).
KE = list(W = env_kernel(env.data = temp, gaussian = FALSE)[[2]])
KE = list(W = env_kernel(env.data = temp)[[2]])

In [ ]:
KE

In [ ]:
## upload manually kinship.RData to /content/ - will be deleted each time
load('/content/kinship.RData')
dim(kk)

In [ ]:
KG = list(G = kk);

In [ ]:
gid = "sire"
y = "milk"
env = "environment"

### Model 1: No envirotyping

Only fixed effects for environment

In [ ]:
M0 = get_kernel(K_G = KG, y = y, gid = gid, env = env, data = Y, model = "MDs")

In [ ]:
fixed = model.matrix(~0+environment, Y)
head(fixed)

In [ ]:
iter = 1000
burn = 200
thin = 10

In [ ]:
fit <- kernel_model(data = Y, y = y, env = env, gid = gid,
random = M0, fixed = fixed, iterations = iter,
burnin = burn, thining = thin)

In [ ]:
fit$VarComp

In [ ]:
M1 = get_kernel(K_G = KG, K_E = KE, y = y, gid = gid, env = env, data = Y, model = "RNMDs")

In [ ]:
fit1 <- kernel_model(data = Y, y = y, env = env, gid = gid,
random = M1, fixed = fixed, iterations = iter,
burnin = burn, thining = thin)

In [ ]:
fit1$VarComp

## Cross-validation

In [ ]:
test_prop = 0.2
n = nrow(Y)
test_size = n*(test_prop)

#### Model 0

In [ ]:
yNA <- Y
tst <- sample(1:n, size=test_size, replace=FALSE)

In [ ]:
yNA[tst, "milk"] <- NA

In [ ]:
head(yNA)

In [ ]:
fit <- kernel_model(data = yNA, y = y, env = env, gid = gid,
random = M0, fixed = fixed, iterations = iter,
burnin = burn, thining = thin)

In [ ]:
cor(Y$milk[-tst], fit$yHat[-tst], use = 'complete.obs')

In [ ]:
cor(Y$milk[tst], fit$yHat[tst], use = 'complete.obs')

In [ ]:
fit$VarComp

#### Model 1

In [ ]:
yNA <- Y
tst1 <- sample(1:n, size=test_size, replace=FALSE)
yNA[tst1, "milk"] <- NA

In [ ]:
fit1 <- kernel_model(data = yNA, y = y, env = env, gid = gid,
random = M1, fixed = fixed, iterations = iter,
burnin = burn, thining = thin)

In [ ]:
cor(Y$milk[-tst1], fit1$yHat[-tst1], use = 'complete.obs')
cor(Y$milk[tst1], fit1$yHat[tst1], use = 'complete.obs')

### Save results

In [ ]:
to_save = list('M0' = fit, 'test0' = tst, 'M1' = fit1, 'test1' = tst1)
save(to_save, file = "/content/results_EnvRtype.RData")